In [1]:
# %pip install -q -U bitsandbytes
# %pip install -q -U git+https://github.com/huggingface/transformers.git
# %pip install -q -U git+https://github.com/huggingface/peft.git
# %pip install -q -U git+https://github.com/huggingface/accelerate.git

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, PeftModel, PeftConfig

In [3]:
base_model_merged = True

In [4]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [6]:
# Load the entire model on the GPU 0
device_map = {"": 0}

In [7]:
# peft_model_id = "../adapters/Mistral-7B-Instruct_finetuned_on_10000_array_basics"
peft_model_id = "../adapters/adapter_from_merged"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, 
                                                    quantization_config=bnb_config, 
                                                    return_dict=True, 
                                                    load_in_4bit=True, 
                                                    device_map={"":0})

if base_model_merged:
    base_model_name = "mistralai/Mistral-7B-Instruct-v0.1"
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
else:
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)


# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
import numpy as np
import random

min_num_rows_and_columns = 1
max_num_rows_and_columns = 8

train_grid_names = [
    ["Train_Input_1", "Train_Output_1"],
    ["Train_Input_2", "Train_Output_2"],
    ["Train_Input_3", "Train_Output_3"],
    ["Train_Input_4", "Train_Output_4"],
    ["Train_Input_5", "Train_Output_5"],
]
test_grid_names = [
    ["Test_Input_1", "Test_Output_1"],
    ["Test_Input_2", "Test_Output_2"],
]

colors = ["black", "blue", "red", "green", "yellow", "gray", "magenta", "orange", "cyan", "brown"]

row_size = random.randint(min_num_rows_and_columns, max_num_rows_and_columns)
column_size = random.randint(min_num_rows_and_columns, max_num_rows_and_columns)
random_array = np.random.randint(0, 10, size=(row_size, column_size))

num_rows = len(random_array)
num_columns = len(random_array[0])

# Display grid in better visual format
print("\nVisualization of grid:\n")
grid = ""
for i in range(num_rows):
    for j in range(num_columns):
        grid += str(random_array[i][j])
    grid += "\n"
print(grid)

train_or_test = random.randint(0, 1)
if train_or_test == 0:
    grid_name = test_grid_names[random.randint(0, len(test_grid_names) - 1)][
        random.randint(0, len(test_grid_names[0]) - 1)
    ]
else:
    grid_name = train_grid_names[random.randint(0, len(train_grid_names) - 1)][
        random.randint(0, len(train_grid_names[0]) - 1)
    ]

random_row = random.randint(0, num_rows - 1)
random_column = random.randint(0, num_columns - 1)

instruction = ""
# random_instruction_intro = random.randint(0, 1)
# if random_instruction_intro == 0:
instruction += (
    "An ARCSolver grid is a rectangular 2D array of integers from 0 to 9. "
)
# instruction += "An ARCSolver grid's integers represent the colors of each grid square rather than scalar values. "
# instruction += "0 = black, 1 = blue, 2 = red, 3 = green, 4 = yellow, 5 = gray, 6 = magenta, 7 = orange, 8 = cyan, 9 = brown. "
# instruction += "Visualize the grid as a flat surface with an x dimension equal the number of columns in centimeters and a y dimension equal to the number of rows in centimeters. "
# instruction += "Therefore, each grid square is a 1cm x 1cm square. "
# instruction += "If the integer in the grid square is a integer from 1 to 9, the grid square is occupied by a 1cm x 1cm tile which is the color that corresponds to its integer. "
# instruction += "The grid surface is painted black. If the integer in the grid square is 0 (black), the grid square is black, and empty. "
# instruction += "If a grid is designated as an ARCSolver grid, All of the rule above apply. "
    
instruction += "If ARCSolver grid "
instruction += grid_name + " = "

instruction += "["
for i in range(num_rows):
    if i == 0:
        instruction += "["
    for j in range(num_columns):
        instruction += str(random_array[i][j])
        if j < num_columns - 1:
            instruction += ","
        else:
            instruction += "]"
    if i < num_rows - 1:
        instruction += ",["
    else:
        instruction += "]"

instruction += ", "

random_question = random.randint(0, 1)
# if random_question == 0:
#     # Row question
#     instruction += "what is " + grid_name + "[" + str(random_row) + "]?"
#     output = grid_name + "[" + str(random_row) + "] is Row " + str(random_row+1) + ", which is the following array: ["
#     for i in range(num_columns):
#         output += str(random_array[random_row][i])
#         if i < num_columns - 1:
#             output += ","
#         else:
#             output += "]"
#     output += "."
# elif random_question == 1:
#     # Column question
#     instruction += "what is Column " + str(random_column+1) + " in " + grid_name + "?"
#     output = "Column " + str(random_column+1) + " in " + grid_name + \
#         " is the integer at index " + str(random_column) + " in each row, which is the following array: ["
#     for i in range(num_rows):
#         output += str(random_array[i][random_column])
#         if i < num_rows - 1:
#             output += ","
#         else:
#             output += "]"
#     output += "."
if random_question == 0:
    # Num rows question
    instruction += "how many rows does " + grid_name + " have?"
    output = grid_name + " has " + str(num_rows) + " rows."
# elif random_question == 2:
#     # Num columns question
#     instruction += "how many columns does " + grid_name + " have?"
#     output = "Each row has " + str(num_columns) + " elements, so "
#     output += grid_name + " has " + str(num_columns) + " columns."
# elif random_question == 4:
#     # Color question
#     instruction += "what color is the ARCSolver grid square at " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]?"
#     output = grid_name + "[" + str(random_row) + "][" + str(random_column) + "] is integer " + \
#         str(random_array[random_row][random_column]) + ", and integer " + \
#             str(random_array[random_row][random_column]) + " = " + \
#                 colors[random_array[random_row][random_column]] + " in an ARCSolver grid. "
#     if colors[random_array[random_row][random_column]] == "black":
#         output += "Since the color is black, the grid square is empty. "
#     else:
#         output += "Since the color is " + colors[random_array[random_row][random_column]] + \
#             ", the grid square is occupied by a(n) " + colors[random_array[random_row][random_column]] + " 1cm x 1cm tile."
# elif random_question == 5:
#     # Right question
#     instruction += "what are the grid squares to the right of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]?"
#     if random_column == num_columns - 1:
#         output = "There are no grid squares to the right of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]."
#     else:
#         output = "The grid squares to the right of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "] are: "
#         for i in range(random_column+1, num_columns):
#             output += grid_name + "[" + str(random_row) + "][" + str(i) + "]"
#             if i < num_columns - 1:
#                 output += ", "
#         output += "."
# elif random_question == 6:
#     # Left question
#     instruction += "what are the grid squares to the left of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]?"
#     if random_column == 0:
#         output = "There are no grid squares to the left of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]."
#     else:    
#         output = "The grid squares to the left of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "] are: "
#         for i in range(random_column-1, -1, -1):
#             output += grid_name + "[" + str(random_row) + "][" + str(i) + "]"
#             if i > 0:
#                 output += ", "
#         output += "."
# elif random_question == 7:
#     # Above question
#     instruction += "what are the grid squares above " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]?"
#     if random_row == 0:
#         output = "There are no grid squares above " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]."
#     else:
#         output = "The grid squares above " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "] are: "
#         for i in range(random_row-1, -1, -1):
#             output += grid_name + "[" + str(i) + "][" + str(random_column) + "]"
#             if i > 0:
#                 output += ", "
#         output += "."
# elif random_question == 8:
#     # Below question
#     instruction += "what are the grid squares below " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]?"
#     if random_row == num_rows - 1:
#         output = "There are no grid squares below " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]."
#     else:
#         output = "The grid squares below " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "] are: "
#         for i in range(random_row+1, num_rows):
#             output += grid_name + "[" + str(i) + "][" + str(random_column) + "]"
#             if i < num_rows - 1:
#                 output += ", "
#         output += "."
# elif random_question == 9:
#     # Diagonally below and to the right question
#     instruction += "what are the grid squares that are diagonally below and to the right of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]?"
#     if random_row == num_rows - 1 or random_column == num_columns - 1:
#         output = "There are no grid squares that are diagonally below and to the right of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]."
#     else:
#         output = "The grid squares that are diagonally below and to the right of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "] are: "
#         for i in range(1, min(num_rows - random_row, num_columns - random_column)):
#             output += grid_name + "[" + str(random_row+i) + "][" + str(random_column+i) + "]"
#             if i < min(num_rows - random_row, num_columns - random_column) - 1:
#                 output += ", "
#         output += "."
# elif random_question == 10:
#     # Diagonally below and to the left question
#     instruction += "what are the grid squares that are diagonally below and to the left of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]?"
#     if random_row == num_rows - 1 or random_column == 0:
#         output = "There are no grid squares that are diagonally below and to the left of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]."
#     else:
#         output = "The grid squares that are diagonally below and to the left of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "] are: "
#         for i in range(1, min(num_rows - random_row, random_column+1)):
#             output += grid_name + "[" + str(random_row+i) + "][" + str(random_column-i) + "]"
#             if i < min(num_rows - random_row, random_column+1) - 1:
#                 output += ", "
#         output += "."
# elif random_question == 11:
#     # Diagonally above and to the right question
#     instruction += "what are the grid squares that are diagonally above and to the right of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]?"
#     if random_row == 0 or random_column == num_columns - 1:
#         output = "There are no grid squares that are diagonally above and to the right of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]."
#     else:
#         output = "The grid squares that are diagonally above and to the right of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "] are: "
#         for i in range(1, min(random_row+1, num_columns - random_column)):
#             output += grid_name + "[" + str(random_row-i) + "][" + str(random_column+i) + "]"
#             if i < min(random_row+1, num_columns - random_column) - 1:
#                 output += ", "
#         output += "."
# elif random_question == 12:
#     # Diagonally above and to the left question
#     instruction += "what are the grid squares that are diagonally above and to the left of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]?"
#     if random_row == 0 or random_column == 0:
#         output = "There are no grid squares that are diagonally above and to the left of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "]."
#     else:
#         output = "The grid squares that are diagonally above and to the left of " + grid_name + "[" + str(random_row) + "][" + str(random_column) + "] are: "
#         for i in range(1, min(random_row+1, random_column+1)):
#             output += grid_name + "[" + str(random_row-i) + "][" + str(random_column-i) + "]"
#             if i < min(random_row+1, random_column+1) - 1:
#                 output += ", "
#         output += "."
else:
    # Square question
    instruction += (
        "what integer is "
        + grid_name
        + "["
        + str(random_row)
        + "]["
        + str(random_column)
        + "]?"
    )
    output = grid_name + "[" + str(random_row) + "][" + str(random_column) + "] is the integer " + str(random_array[random_row][random_column]) + "."

ground_truth = output

query = instruction
prompt = '<s>[INST] ' + query + ' [/INST] '
print("Query:\n", prompt)
encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to("cuda")

# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# model.resize_token_embeddings(len(tokenizer))
# tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=False, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
result = decoded[0]
print("\nResult:\n", result)

print("\nGround truth:\n", ground_truth)

# Tokenize the request text
tokenized_request = tokenizer.tokenize(result)

# Get the token length
token_length = len(tokenized_request)

print("\nNumber of tokens: ", token_length)


Visualization of grid:

978066
537831
327237
656765

Query:
 <s>[INST] An ARCSolver grid is a rectangular 2D array of integers from 0 to 9. If ARCSolver grid Test_Input_1 = [[9,7,8,0,6,6],[5,3,7,8,3,1],[3,2,7,2,3,7],[6,5,6,7,6,5]], how many rows does Test_Input_1 have? [/INST] 

Result:
 <s> [INST] An ARCSolver grid is a rectangular 2D array of integers from 0 to 9. If ARCSolver grid Test_Input_1 = [[9,7,8,0,6,6],[5,3,7,8,3,1],[3,2,7,2,3,7],[6,5,6,7,6,5]], how many rows does Test_Input_1 have? [/INST] 4 rows.</s>

Ground truth:
 Test_Input_1 has 4 rows.

Number of tokens:  111
